In [1]:
import pandas as pd
df=pd.read_csv("UNSW_NB15_testing-set.csv")

In [2]:
df.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0


In [3]:
from sklearn.preprocessing import LabelEncoder

In [4]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Initialize LabelEncoder
le = LabelEncoder()

# Fit on attack_cat column
le.fit(df['attack_cat'])

# Get all classes
classes = list(le.classes_)

# Move 'Normal' to index 0
if 'Normal' in classes:
    classes.remove('Normal')
    classes = ['Normal'] + classes  # Normal first

# Create a mapping dictionary
label_mapping = {label: idx for idx, label in enumerate(classes)}

# Map attack_cat to numerical labels
df['label'] = df['attack_cat'].map(label_mapping)

print(label_mapping)
print(df[['attack_cat', 'label']].head())


{'Normal': 0, 'Analysis': 1, 'Backdoor': 2, 'DoS': 3, 'Exploits': 4, 'Fuzzers': 5, 'Generic': 6, 'Reconnaissance': 7, 'Shellcode': 8, 'Worms': 9}
  attack_cat  label
0     Normal      0
1     Normal      0
2     Normal      0
3     Normal      0
4     Normal      0


In [5]:
#check for dupilicates
print("Test duplicates:", df.duplicated().sum())

Test duplicates: 0


In [6]:
#save to CSV
df.to_csv("dataset_label_test.csv", index=False)

In [7]:
df.drop('attack_cat', axis=1, inplace=True)


In [8]:
# Drop id
df.drop("id",axis=1,inplace=True)

In [9]:
df.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
0,0.000011,udp,-,INT,2,0,496,0,90909.0902,254,...,1,1,2,0,0,0,1,2,0,0
1,0.000008,udp,-,INT,2,0,1762,0,125000.0003,254,...,1,1,2,0,0,0,1,2,0,0
2,0.000005,udp,-,INT,2,0,1068,0,200000.0051,254,...,1,1,3,0,0,0,1,3,0,0
3,0.000006,udp,-,INT,2,0,900,0,166666.6608,254,...,2,1,3,0,0,0,2,3,0,0
4,0.000010,udp,-,INT,2,0,2126,0,100000.0025,254,...,2,1,3,0,0,0,2,3,0,0


In [10]:
#check for dupilicates
print("Duplicates:",df.duplicated().sum())

Duplicates: 26387


In [11]:
df.drop_duplicates(inplace=True)


In [12]:
#check for dupilicates
print("Duplicates:",df.duplicated().sum())

Duplicates: 0


In [13]:
df = df.sample(frac=1, random_state=42)
df.reset_index(drop=True, inplace=True)


In [14]:
#check for categorical columns
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
print("Categorical columns (by dtype):", categorical_columns.tolist())

Categorical columns (by dtype): ['proto', 'service', 'state']


In [15]:
import pickle
import joblib
# Load scaler
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)


In [16]:
encoders = joblib.load('label_encoders.pkl')

In [17]:
print(encoders)

{'proto': LabelEncoder(), 'service': LabelEncoder(), 'state': LabelEncoder()}


In [18]:
X = df.drop('label', axis=1)
y = df['label']


In [19]:

duplicates_mask = X.duplicated(keep='first')


In [20]:
X = X[~duplicates_mask].reset_index(drop=True)


In [21]:
y = y[~duplicates_mask].reset_index(drop=True)


In [22]:
#check for dupilicates
print("Duplicates:",X.duplicated().sum())

Duplicates: 0


In [23]:
import pandas as pd

# Categorical columns
categorical_cols = ['proto', 'service', 'state']

for col, le in encoders.items():
    # Replace unseen categories
    X[col] = X[col].map(lambda s: s if s in le.classes_ else 'Unknown')

    # Extend encoder to handle 'Unknown' category if not already present
    if 'Unknown' not in le.classes_:
        le.classes_ = np.append(le.classes_, 'Unknown')

    # Transform test data using same mapping
    X[col] = le.transform(X[col])

In [24]:
#check for infinite values
import numpy as np

# Select only numeric columns
X_numeric = X.select_dtypes(include=np.number)

# Check for infinite values in numeric columns
print(np.isinf(X_numeric).sum().sum())

0


In [25]:
#check for null values
#print(df.isnull().sum())
#print(df.isnull().any())
print("Total nulls:", X.isnull().values.sum())


Total nulls: 0


In [26]:
X = X.dropna().reset_index(drop=True)


In [27]:
y

0        6
1        4
2        0
3        0
4        0
        ..
53941    0
53942    0
53943    7
53944    7
53945    0
Name: label, Length: 53946, dtype: int64

In [28]:
# Check for placeholder or empty-string values
for col in X.columns:
    unique_vals = X[col].unique()
    if '-' in unique_vals or '' in unique_vals:
        print(f"Column {col} has '-' or empty values")

In [29]:
#feature scaling
X_test = scaler.transform(X)

In [30]:
y = y.astype(int)


In [31]:
# Get column names
feature_names = X.columns

# Create DataFrames from the scaled numpy arrays
X_test_df = pd.DataFrame(X_test, columns=feature_names)

# Concatenate the DataFrames with the target Series
test_df = pd.concat([X_test_df, y.reset_index(drop=True)], axis=1)

test_df.to_csv("pretest_dataset.csv", index=False)


In [32]:
test_df.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
0,1.833334e-07,0.901515,0.166667,0.375,0.000104,0.000000,0.000007,0.000000,0.090909,0.996078,...,0.04,0.044444,0.031250,0.0,0.0,0.000000,0.033898,0.032787,0.0,6
1,2.150927e-02,0.856061,0.416667,0.250,0.000936,0.001093,0.000059,0.000504,0.000016,0.243137,...,0.00,0.000000,0.000000,0.0,0.0,0.033333,0.000000,0.016393,0.0,4
2,7.886585e-03,0.856061,0.000000,0.000,0.000520,0.000182,0.000075,0.000006,0.000015,0.243137,...,0.00,0.000000,0.078125,0.0,0.0,0.000000,0.016949,0.081967,0.0,0
3,2.164929e-02,0.856061,0.750000,0.250,0.005304,0.003463,0.002869,0.000216,0.000069,0.121569,...,0.02,0.000000,0.015625,0.0,0.0,0.000000,0.033898,0.032787,0.0,0
4,5.000001e-08,0.856061,0.000000,0.625,0.000104,0.000000,0.000005,0.000000,0.333333,0.996078,...,0.00,0.000000,0.140625,0.0,0.0,0.000000,0.033898,0.147541,0.0,0


In [33]:
y.value_counts()

label
0    34203
4     7202
5     4439
6     3642
7     2491
3     1297
8      378
1      134
2      116
9       44
Name: count, dtype: int64

In [34]:
#check for dupilicates
print("Test duplicates:", test_df.duplicated().sum())

Test duplicates: 0
